# Import Libraries

In [ ]:

import os
from torchvision.transforms import Compose, Resize, InterpolationMode, ToTensor, Normalize, RandomCrop, RandomHorizontalFlip
import torch
from code.classifier import PneumoniaClassifier, Config
from code.plot_random_image import plot_random_image_from_loader
from code.classify_random_images import plot_random_images_with_labels
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix
import matplotlib.pyplot as plt
from torchvision import transforms
from code.classifier import PneumoniaClassifier
from code.dataloader import DataLoader



In [ ]:

# Load the TensorBoard
%reload_ext tensorboard
%load_ext tensorboard


In [ ]:
# Enable CUDA Launch Blocking
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
# Set float32 matrix multiplication precision to 'medium' to utilize Tensor Cores
torch.set_float32_matmul_precision('medium')
torch.serialization.add_safe_globals([Compose, Resize, set, Config, InterpolationMode, ToTensor, Normalize, RandomHorizontalFlip])

# Check GPU Availability

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
    print('Not connected to a GPU')
else:
    print(gpu_info)

# Setup Model

In [ ]:
config = Config(
    backbone_name="google/vit-base-patch16-224",
    transfer_learning=True,
    learning_rate=1e-5,
    batch_size=32,
    max_epochs=100,
    weight_decay=1e-5,
    dropout=0.3,
    num_workers=16,
    model_name="Vit-unfreeze",
    version="001",
    optimizer_name = "adam",
    use_class_weights=True,
    image_res=384,
    patience = 10,
    image_type=3
)
model = PneumoniaClassifier(config)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)




# Plot random images from all datasets

In [ ]:
plot_random_image_from_loader(model.train_loader.dataset, "Train Dataset", normalize=False)

In [ ]:
plot_random_image_from_loader(model.val_loader.dataset, "Validation Dataset", normalize=False)

In [ ]:
plot_random_image_from_loader(model.test_loader.dataset, "Test Dataset", normalize=False)

# Train Model

In [ ]:

# Train Model
model.train_model()
torch.save({
    "state_dict": model.state_dict(),
    "config": config
}, f"../models/{config.model_name}_final.pt")

# Load Model

In [ ]:

checkpoint = torch.load(f"../models/{config.model_name}_final.pt")
loaded_config = checkpoint["config"]
model = PneumoniaClassifier(loaded_config)
model.load_state_dict(checkpoint["state_dict"])
model.eval()
model = model.to(device)

# Test Model

In [ ]:
# Run testing
print("Testing the model...")
checkpoint_path = f"../checkpoints/{config.model_name}.ckpt"
test_metadata = model.test_model(checkpoint_path)  # Ensure this uses the correct test_loader

# Plot Confusion Matrix

In [ ]:
def plot_confusion_matrix(model, dataloader, class_names):
    all_preds = []
    all_labels = []

    # Get predictions and true labels
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    model.eval()
    with torch.no_grad():
        for data, labels, _ in dataloader:
            data = data.cuda()  # Move data to GPU if available
            labels = labels.cuda()
            preds = torch.argmax(model(data), dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # Compute confusion matrix
    cm = confusion_matrix(all_labels, all_preds)
    disp = ConfusionMatrixDisplay(cm, display_labels=class_names)
    disp.plot(cmap="Blues")
    plt.title("Confusion Matrix")
    plt.show()

# Call this after training/testing
plot_confusion_matrix(model, model.test_loader, class_names=["Normal", "Pneumonia"])

# Plot ROC/AUC Score

In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score

def plot_roc_auc(model, dataloader):
    all_preds = []
    all_labels = []

    # Get predictions and true labels
    model.eval()
    with torch.no_grad():
        for data, labels, _ in dataloader:
            data = data.cuda()
            labels = labels.cuda()
            preds = model(data)
            probs = torch.softmax(preds, dim=1)[:, 1]  # Probabilities for class 1
            all_preds.extend(probs.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # Compute ROC curve and AUC
    fpr, tpr, _ = roc_curve(all_labels, all_preds)
    auc = roc_auc_score(all_labels, all_preds)

    # Plot ROC curve
    plt.figure()
    plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {auc:.2f})')
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve')
    plt.legend(loc="lower right")
    plt.show()

plot_roc_auc(model, model.test_loader)

# Plot Gradcam

In [ ]:
for param in model.feature_extractor.parameters():
    param.requires_grad = True

from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.image import show_cam_on_image
import numpy as np

def reshape_transform(tensor):
    activations = tensor[:, 1:, :]
    activations = activations.reshape(activations.shape[0], int(np.sqrt(activations.shape[1])), -1, activations.shape[2])
    return activations.permute(0, 3, 1, 2)

# Use the last layer of the encoder as the target
target_layer = model.feature_extractor.vit.encoder.layer[-1]

# Initialize Grad-CAM
cam = GradCAM(model=model, target_layers=[target_layer], reshape_transform=reshape_transform)

# Pass input image tensor
input_tensor = inputs  # Your input tensor
grayscale_cam = cam(input_tensor=input_tensor)

# Visualize Grad-CAM
heatmap = show_cam_on_image(rgb_image, grayscale_cam[0], use_rgb=True)



# Evaluate Metrics in Tensorboard

In [ ]:
# Load TensorBoard extension
%load_ext tensorboard

# Specify the log directory
log_dir = f"tb_logs/{config.model_name}"

# Start TensorBoard
%tensorboard --logdir {log_dir}

# Classify random images

In [ ]:
plot_random_images_with_labels(model, model.test_loader, num_images=20)